# Amazon Bedrock AgentCore Gateway - Semantic search 튜토리얼

### 튜토리얼 세부사항


| 정보         | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 대화형                                                                   |
| Agent 유형          | 단일                                                                           |
| AgentCore 서비스  | AgentCore Gateway, AgentCore Identity                                            |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                        |
| 튜토리얼 구성요소 | Strands Agent에서 Lambda 기반 AgentCore Gateway 생성 및 사용            |
| 튜토리얼 분야   | 범용                                                                   |
| 예제 복잡도  | 쉬움                                                                             |
| 사용된 SDK            | Amazon BedrockAgentCore Python SDK 및 boto3  

### 튜토리얼 아키텍처
Amazon Bedrock AgentCore Gateway는 agent와 상호작용해야 하는 tool 및 리소스 간의 통합 연결을 제공합니다. Gateway는 이 연결 계층에서 여러 역할을 수행합니다:

1. **Security Guard**: Gateway는 OAuth 인증을 관리하여 유효한 사용자/agent만 tool/리소스에 액세스할 수 있도록 합니다.
2. **Translator**: Gateway는 Model Context Protocol(MCP)과 같은 인기 있는 프로토콜을 사용하여 만든 agent 요청을 API 요청 및 Lambda 호출로 변환합니다. 이는 개발자가 서버를 호스팅하거나 프로토콜 통합, 버전 지원, 버전 패치 등을 관리할 필요가 없음을 의미합니다.
3. **Composer**: Gateway를 사용하면 개발자가 여러 API, 함수 및 tool을 agent가 사용할 수 있는 단일 MCP 엔드포인트로 원활하게 결합할 수 있습니다.
4. **Keychain**: Gateway는 올바른 tool과 함께 사용할 올바른 자격 증명의 주입을 처리하여 agent가 다양한 자격 증명 세트가 필요한 tool을 원활하게 활용할 수 있도록 합니다. 
5. **Researcher**: Gateway를 사용하면 agent가 모든 tool을 검색하여 특정 컨텍스트나 질문에 가장 적합한 tool만 찾을 수 있습니다. 이를 통해 agent는 소수가 아닌 수천 개의 tool을 사용할 수 있습니다. 또한 agent의 LLM 프롬프트에 제공해야 하는 tool 세트를 최소화하여 지연 시간과 비용을 줄입니다. 
6. **Infrastructure Manager**: Gateway는 완전히 서버리스이며 내장된 observability 및 감사 기능을 제공하여 개발자가 agent와 tool을 통합하기 위해 추가 인프라를 관리할 필요가 없습니다. 

![How does it work](images/gw-arch-overview.png)

### 튜토리얼 주요 기능

* AWS Lambda 기반 target으로 Amazon Bedrock AgentCore Gateway 생성
* AgentCore Gateway semantic search 사용 
* Strands Agents를 사용하여 AgentCore Gateway search가 지연 시간을 개선하는 방법 시연


## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents

## AgentCore Gateway는 다수의 tool을 가진 MCP 서버의 문제 해결을 지원합니다
일반적인 엔터프라이즈 환경에서 agent 개발자는 수백 또는 수천 개의
MCP tool을 가진 MCP 서버를 접하게 됩니다. 이러한 대량의 tool은 AI agent에게 tool 선택 정확도 저하, 
비용 증가, 과도한 tool 메타데이터로 인한 높은 토큰 사용량으로 인한 지연 시간 증가 등의 문제를 야기합니다.
이는 agent를 타사 서비스(예: Zendesk, Salesforce,
Slack, JIRA 등) 또는 기존 엔터프라이즈 REST 서비스에 연결할 때 발생할 수 있습니다. 
AgentCore Gateway는 tool 전반에 걸친 내장 semantic search를 제공하여 
agent의 지연 시간, 비용 및 정확도를 개선하면서도 agent에게 필요한 tool을 제공합니다. 
사용 사례, LLM model 및 agent framework에 따라 일반적인 MCP Server의 수백 개 tool 전체 세트를 제공하는 것과 비교하여
관련 tool에 집중하는 agent를 유지함으로써 최대 3배 더 나은 지연 시간을 확인할 수 있습니다.

![How does it work](images/gateway_tool_search.png)

## 이 노트북에서 배울 내용
이 노트북에서는 AgentCore Gateway search에 대한 튜토리얼을 제공합니다. 이 단계별 튜토리얼을 마치면
다음을 이해하게 됩니다:

- AgentCore Gateway의 내장 search tool을 사용하여 관련 tool을 빠르게 찾는 방법 
- 지연 시간 개선 및 비용 절감을 위해 tool 검색 결과를 Strands Agents에 통합하는 방법

## 노트북 구조 개요
노트북은 다음 섹션으로 구성되어 있습니다:

1. AgentCore Gateway Search의 기초 이해
2. 노트북 환경 준비
3. 수백 개의 tool을 가진 Gateway 설정
4. Gateway에서 tool 검색
5. 많은 tool을 가진 MCP 서버와 Strands Agents 사용
6. Strands Agent에 tool 검색 결과 추가
7. Showing 3x latency improvement by using tool search

# AgentCore Gateway Search의 기초 이해

AgentCore Gateway를 생성할 때 Search를 활성화할 것인지 지정할 수 있습니다.
Search가 활성화된 Gateway의 경우 다음 세 가지가 발생합니다:

1. **Vector store가 생성됩니다**. Gateway 서비스는 새 Gateway에 대해 서버리스 완전 관리형 vector store를 자동으로 생성합니다. 이를 통해 Gateway tool 전반에 걸친 완전한 semantic search가 가능합니다. 
3. **Vector store가 채워집니다**. Gateway에 Gateway Target을 추가하면 서비스는 자동으로 백그라운드에서 embedding을 사용하여 새 Target의 tool을 기반으로 vector store를 채웁니다. tool 메타데이터는 tool의 JSON 정의 또는 REST 서비스 target에 대한 OpenAPI Schema 사양에서 가져옵니다.
2. **Search tool (MCP based) is provided**. In addition to all of your user defined tools (from AWS Lambda targets or REST services), the Gateway gets one additional MCP tool that provides semantic search. It is named `x-amz-bedrock-agentcore-search`. The prefix ensures there are no name clashes with your user-defined tools. We may add more tools like that in the future as well. The search tool has a single argument called `query`. When the search tool is invoked, the Gateway service performs a semantic search using that query, matching it against available tool metadata (names, descriptions, input and output schema), and returns the most relevant tools in descending order of relevance.

# 노트북 환경 준비


In [ ]:
# 필요한 패키지 강제 재설치 및 업그레이드
!pip install --force-reinstall -U -r requirements.txt --quiet

Import all the required Python libraries, and load environment variables

In [ ]:
# Strands Agent framework 및 Bedrock model import
from strands import Agent
from strands.models import BedrockModel
from strands.handlers import null_callback_handler

# MCP (Model Context Protocol) 클라이언트 및 tool import
from strands.tools.mcp.mcp_client import MCPClient, MCPAgentTool

# MCP HTTP 클라이언트
from mcp.client.streamable_http import streamablehttp_client
from mcp.types import Tool as MCPTool

import logging
import time
import json
import boto3
import requests
import utils

GATEWAY_NAME = "gateway-search-tutorial"

Set up a logger

In [ ]:
# Strands logger 설정 (ERROR 레벨로 설정하여 불필요한 로그 숨김)
logging.getLogger("strands").setLevel(logging.ERROR)  # INFO) #DEBUG) #

# 로그 출력 형식 설정
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()]
)

Check our boto3 version

In [ ]:
boto3.__version__

Get our boto3 client for the AgentCore control plane API.

In [ ]:
# boto3 세션 생성 및 AgentCore control plane API 클라이언트 초기화
session = boto3.Session()
agentcore_client = session.client(
    "bedrock-agentcore-control",
)

# 수백 개의 tool을 가진 Gateway 설정

AgentCore Gateway는 기존 API의 선별된 세트를
agent용 MCP tool로 노출하는 안전하고 확장 가능한 방법을 제공합니다. 프로덕션 환경에서는 Gateway 리소스가
CloudFormation, CDK 또는 Terraform과 같은 도구를 사용하여 infrastructure as code로 생성됩니다. 이 튜토리얼에서는
boto3 control plane API를 직접 사용하여 리소스와 API를 더 효과적으로 이해할 수 있도록 합니다.
이를 통해 자신만의 gateway를 더 쉽게 구축하고 사용하며 더 강력하고
안전한 agent를 만들 수 있습니다.

높은 수준에서 Gateway 설정 단계는 다음과 같습니다:

1. 인바운드(Gateway를 호출하는 agent) 및 아웃바운드(tool을 호출하는 Gateway) 보안에 사용할 identity provider 및 credential provider를 정의합니다.
2. `create_gateway`를 사용하여 Gateway를 생성합니다.
3. `create_gateway_target`를 사용하여 Gateway Target을 추가하여 AWS Lambda 또는 기존 RESTful 서비스에서 구현될 MCP tool을 노출합니다.

In this tutorial, we will use Amazon Cognito as the identity provider (IdP), AWS Lambda functions as targets, and AWS IAM for outbound authentication. The same concepts demonstrated in this tutorial still apply when using other IdP's or other target types.

### Amazon Cognito 리소스 생성


이 튜토리얼에서는 다음 리소스를 이미 생성하고 해당 환경 변수를 설정했다고 가정합니다:

- AWS Lambda 실행을 위한 IAM role (`gateway_lambda_iam_role`)
- 간단한 수학 tool을 위한 AWS Lambda 함수 (`calc_lambda_arn`)
- 레스토랑 예약 tool을 위한 AWS Lambda 함수 (`restaurant_lambda_arn`)
- Amazon Cognito user pools, giving you a client id (`cognito_client_id`) and a discovery URL (`cognito_discovery_url`)

Lets take a look at the JSON tool metadata for the restaurant API. Note that if we were integrating with existing REST services, the API specs would be provided using OpenAPI Schema instead.

In [ ]:
with open("./restaurant/restaurant-api.json") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

Here are the simple calculator APIs.

In [ ]:
with open("./calc/calc-api.json") as f:
    data = json.load(f)[0:3]
print(json.dumps(data, indent=4))

Here is the AWS Lambda function implementation for the calculator tools.

In [ ]:
from IPython.display import display, Code

with open("./calc/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
with open("./restaurant/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
# 계산기 Lambda 함수 생성
calc_lambda_resp = utils.create_gateway_lambda(
    "calc/lambda_function_code.zip", lambda_function_name="calc_lambda_gateway"
)

if calc_lambda_resp is not None:
    if calc_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            calc_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            calc_lambda_resp["lambda_function_arn"],
        )

In [ ]:
calc_lambda_resp["lambda_function_arn"]

In [ ]:
# 레스토랑 예약 Lambda 함수 생성
restaurant_lambda_resp = utils.create_gateway_lambda(
    "restaurant/lambda_function_code.zip",
    lambda_function_name="restaurant_lambda_gateway",
)

if restaurant_lambda_resp is not None:
    if restaurant_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )

In [ ]:
restaurant_lambda_resp["lambda_function_arn"]

In [ ]:
# Cognito user pool 설정 (OAuth 인증용)
cognito_response = utils.setup_cognito_user_pool()

In [ ]:
# Cognito에서 JWT bearer token 획득
bearer_token = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)

In [ ]:
# Gateway가 Lambda를 호출할 수 있는 IAM role 생성
gateway_role_arn = utils.create_gateway_iam_role(
    lambda_arns=[
        calc_lambda_resp["lambda_function_arn"],
        restaurant_lambda_resp["lambda_function_arn"],
    ]
)

#### control plane API 사용을 위한 몇 가지 헬퍼 함수 생성


In [ ]:
def read_apispec(json_file_path):
    try:
        # JSON 파일을 읽어 Python 객체로 파싱
        with open(json_file_path, "r") as file:
            api_spec = json.load(file)
            return api_spec

    except FileNotFoundError:
        return f"Error: File {json_file_path} not found"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"


def list_gateways():
    # 모든 gateway 목록 조회
    response = agentcore_client.list_gateways()
    print(json.dumps(response, indent=2, default=str))
    return response

#### Gateway 생성 헬퍼 함수
이름과
설명이 주어진 AgentCore Gateway를 생성하기 위한 헬퍼 함수입니다. Amazon Cognito를 IdP로 사용하고 허용된 client ID와
discovery URL을 환경 변수에서 가져옵니다. 이미 정의되어 있기 때문입니다.
또한 결과 Gateway에서 semantic search를 기본적으로 활성화하고
a predefined IAM role.

In [ ]:
def create_gateway(gateway_name, gateway_desc):
    # Cognito를 IdP로 사용하여 Gateway 인바운드 OAuth 설정
    auth_config = {
        "customJWTAuthorizer": {
            "allowedClients": [cognito_response["client_id"]],
            "discoveryUrl": cognito_response["discovery_url"],
        }
    }
    # tool의 semantic search 활성화
    search_config = {
        "mcp": {"searchType": "SEMANTIC", "supportedVersions": ["2025-03-26"]}
    }
    # Gateway 생성
    response = agentcore_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        authorizerType="CUSTOM_JWT",
        description=gateway_desc,
        protocolType="MCP",
        authorizerConfiguration=auth_config,
        protocolConfiguration=search_config,
    )
    return response["gatewayId"]

#### Gateway Target 생성 헬퍼 함수
이 함수는 기존 Gateway에 새 AWS Lambda target을 생성합니다.
gateway ID, 새 target의 이름과 설명,
기존 AWS Lambda 함수의 ARN 및 gateway에서 노출하려는 tool의
the interfaces to the tools you want to expose from the gateway.

In [ ]:
def create_gatewaytarget(gateway_id, target_name, target_descr, lambda_arn, api_spec):
    # Gateway에 Lambda target 추가
    response = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        description=target_descr,
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_arn,
                    "toolSchema": {"inlinePayload": api_spec},
                }
            }
        },
        # IAM을 credential provider로 사용
        credentialProviderConfigurations=[
            {"credentialProviderType": "GATEWAY_IAM_ROLE"}
        ],
    )
    return response["targetId"]

### 첫 번째 AgentCore Gateway 생성
첫 번째 Gateway를 설정하기 전에 
Gateway가 MCP tool을 사용하기 위한 인바운드 요청과
Gateway에서 tool 및 리소스로의 아웃바운드 액세스 모두에 대해 보안을 제공하는 방법을 간단히 살펴보겠습니다.

![How does it work](images/gateway_secure_access.png)

Now let's create the gateway for this tutorial, providing a name and a description.

In [ ]:
print(f"Create gateway with name: {GATEWAY_NAME}")
gatewayId = create_gateway(
    gateway_name=GATEWAY_NAME, gateway_desc="AgentCore Gateway Tutorial"
)
print(f"Gateway created with id: {gatewayId}.")

### AgentCore Gateway Target 추가
이 튜토리얼에서는 간단한
수학 계산을 수행하는 Lambda 함수와 레스토랑 예약 생성을 시뮬레이션하는 또 다른 Lambda 함수 한 쌍을 이미 설치했다고 가정합니다. 각 함수에 대해 Gateway Target을
추가하겠습니다.

이러한 target을 추가한 후에는 AgentCore Gateway search의 강력함을 시연하는 데 도움이 되는
help us demonstrate the power of AgentCore Gateway search.

이제 gateway가 생성되었으므로 Lambda 함수를 통해 레스토랑 예약을 만들기 위한
via a Lambda function.

In [ ]:
restaurant_api_spec = read_apispec("./restaurant/restaurant-api.json")
restaurant_lambda_arn = restaurant_lambda_resp["lambda_function_arn"]
print(f"Restaurant Lambda ARN: {restaurant_lambda_arn}")

restaurantTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=restaurant_lambda_arn,
    target_name="FoodTools",
    target_descr="Restaurant Tools",
    api_spec=restaurant_api_spec,
)
print(f"RestaurantTarget created with id: {restaurantTargetId} on gateway: {gatewayId}")

여기서는 4개의 기본 tool(더하기, 빼기,
곱하기, 나누기)을 구현하는 Lambda와 투자 관리(거래, 신용 조사, 정량 분석, 포트폴리오 관리)를 위한 75개의 생성된 tool 정의 세트를 사용하여 두 번째 target을 추가하겠습니다. 투자 관리 tool 정의는 
actually implemented in the Lambda function. We are only adding them to demonstrate a large volume of tools.

In [ ]:
calc_api_spec = read_apispec("./calc/calc-api.json")
print(f"API spec for calc has {len(calc_api_spec)} functions\n")
calc_lambda_arn = calc_lambda_resp["lambda_function_arn"]
print(f"Calc Lambda ARN: {calc_lambda_arn}")

time.sleep(5)
calcTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=calc_lambda_arn,
    target_name="CalcTools",
    target_descr="Calculation Tools",
    api_spec=calc_api_spec,
)
print(f"CalcTools Target created with id: {calcTargetId} on gateway: {gatewayId}")

gateway search의 강력함을 시연하기 위해 이제 Calculator target의 복사본을 몇 개 더 추가하여 
so that we end up with 300+ MCP tools exposed.

In [ ]:
def add_more_tools(gatewayId):
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc2",
        target_descr="Calculation 2 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc2 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc3",
        target_descr="Calculation 3 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc3 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc4",
        target_descr="Calculation 4 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc4 Target created with id: {calcTargetId} on gateway: {gatewayId}")

In [ ]:
add_more_tools(gatewayId=gatewayId)

In [ ]:
resp = agentcore_client.list_gateway_targets(gatewayIdentifier=gatewayId)
targets = resp["items"]
for target in resp["items"]:
    print(f"{target['name']} - {target['description']}")

# Gateway에서 tool 검색


### 검색하기 전에 MCP list tools에 익숙해지기


주어진 Gateway ID에 대한 MCP 엔드포인트 URL을 검색하고 Gateway를 안전하게 사용하기 위한
retrieve our JWT OAuth access token to securely use our Gateway.

In [ ]:
def get_gateway_endpoint(gateway_id):
    response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = response["gatewayUrl"]
    return gateway_url

이제 Gateway가 생성되고 target이 있으므로 해당
Gateway. We can retrieve the endpoint URL from the Gateway control plane based on the Gateway ID.

#### Gateway에 대해 MCP Inspector 사용

이제 MCP 서버에 대한 엔드포인트 URL과 JWT bearer 토큰이 있으므로
MCP Inspector tool로 MCP 서버를 탐색할 수 있습니다. MCP Inspector는 모든 MCP
서버에 연결할 수 있고 제공된 tool을 나열하며 사용하기 쉬운 tool 호출 경험을 제공하는 오픈 소스 tool입니다. 

터미널 창에서 `npx @modelcontextprotocol/inspector`를 입력하여 MCP Inspector를 시작하기만 하면 됩니다. 그런 다음
Gateway 엔드포인트 URL과 JWT 토큰을 붙여넣어 연결합니다. 연결되면 List Tools 및 Invoke Tool을 시도해 보세요.

샘플 스크린샷은 다음과 같습니다.

![MCP Inspector](images/mcp_inspector.png)

In [ ]:
gatewayEndpoint = get_gateway_endpoint(gateway_id=gatewayId)
print(f"Gateway Endpoint - MCP URL: {gatewayEndpoint}")

MCP 서버 보안은 OAuth를 기반으로 합니다. Gateway와 상호작용하려면
retrieve a JWT OAuth access token from our IdP.

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
print(f"Bearer token: {jwtToken}")

In [ ]:
# !npx @modelcontextprotocol/inspector

#### jsonrpc를 사용하여 MCP tool을 호출하거나 나열하는 헬퍼 함수 생성
jsonrpc를 사용하여 Gateway를 포함한 MCP Server가 노출하는 MCP tool 중 하나를 호출하는
`invoke_gateway_tool`이라는 헬퍼 함수를 정의하겠습니다. 엔드포인트 URL과 JWT 토큰이 주어지면
Gateway에 Gateway Target을 추가할 때 AgentCore Gateway가 사용 가능하게 만든
for you when you added Gateway Targets to your Gateway.

In [ ]:
def invoke_gateway_tool(gateway_endpoint, jwt_token, tool_params):
    # MCP jsonrpc 프로토콜을 사용하여 tool 호출
    requestBody = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/call",
        "params": tool_params,
    }
    response = requests.post(
        gateway_endpoint,
        json=requestBody,
        headers={
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        },
    )

    return response.json()

Gateway에서 사용 가능한 MCP tool을 나열하기 위해 MCP의 `tools/list` 메서드를 사용하는
또 다른 유틸리티 함수입니다. Gateway ID와 JWT Token이 주어지면 해당 Gateway에서
전체 tool 세트를 검색하고 agent에서 바로 사용할 수 있는 형태의 목록을 반환합니다. 반환된 목록에는 
Agent에 전달하기에 적합한 Strands Agents MCPAgentTool 객체가 포함되어 있습니다. 

`tools/list` 호출은 페이지네이션되므로 함수는 루프를 돌면서 한 번에 한 페이지의
tool을 가져와야 하며 `nextCursor` 필드가 더 이상 채워지지 않을 때까지 계속됩니다. 유틸리티 함수는
HTTPS 및 jsonrpc 프로토콜을 사용하여 엔드포인트를 직접 호출합니다. 이는 Strands Agents가 제공하는
compared to the `MCPClient` class provided by Strands Agents. We'll see that experience later.

In [ ]:
def get_all_agent_tools_from_mcp_endpoint(gateway_endpoint, jwt_token, client):
    # MCP 엔드포인트에서 페이지네이션을 통해 모든 tool 가져오기
    more_tools = True
    tools_count = 0
    tools_list = []

    requestBody = {"jsonrpc": "2.0", "id": 2, "method": "tools/list", "params": {}}
    next_cursor = ""

    while more_tools:
        if tools_count == 0:
            requestBody["params"] = {}
        else:
            print(f"\nGetting next page of tools since a next cursor was returned\n")
            requestBody["params"] = {"cursor": next_cursor}

        headers = {
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        }

        print(f"\n\nListing tools for gateway {gateway_endpoint}")

        response = requests.post(gateway_endpoint, json=requestBody, headers=headers)

        tools_json = response.json()
        tools_count += len(tools_json["result"]["tools"])

        # MCP tool을 Strands MCPAgentTool로 변환
        for tool in tools_json["result"]["tools"]:
            mcp_tool = MCPTool(
                name=tool["name"],
                description=tool["description"],
                inputSchema=tool["inputSchema"],
            )
            mcp_agent_tool = MCPAgentTool(mcp_tool, client)
            short_descr = tool["description"][0:40] + "..."
            print(f"adding tool '{mcp_agent_tool.tool_name}' - {short_descr}")
            tools_list.append(mcp_agent_tool)

        # nextCursor가 있으면 다음 페이지 가져오기
        if "nextCursor" in tools_json["result"]:
            next_cursor = tools_json["result"]["nextCursor"]
            more_tools = True
        else:
            more_tools = False

    print(f"\nTotal tools found: {tools_count}\n")
    return tools_list

Lets use this helper function and see the results.

In [ ]:
# MCP 클라이언트 생성 (streamable HTTP 사용)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_agent_tools_from_mcp_endpoint(
        gateway_endpoint=gatewayEndpoint, jwt_token=jwtToken, client=client
    )
    print(f"\nFound {len(all_tools)} tools using jsonrpc to list MCP tools\n")

#### 페이지네이션과 함께 Strands Agents list_tools_sync() 사용
Python 기반 MCP 클라이언트를 작성한 적이 있다면 클라이언트가 연결된
MCP Server에서 사용 가능한 tool 세트를 반환하는 `list_tools_sync()` 메서드에
익숙할 것입니다. 그런데 MCP list tools도 페이지네이션된다는 것을 알고 계셨나요? 기본적으로 첫 번째 작은
tool 하위 집합만 반환됩니다. 간단한 MCP 서버의 경우 이를 눈치채지 못했을 수 있지만 많은 실제 
MCP 서버의 경우 코드가 루프를 돌면서 한 번에 tool 페이지를 가져와야 하며
더 이상 남은 페이지가 없을 때까지 계속해야 합니다. 다음 유틸리티 `get_all_mcp_tools_from_mcp_client`가 정확히 그 일을 합니다. 
It returns the full list of tools from a given Strands Agent MCP Client.

In [ ]:
def get_all_mcp_tools_from_mcp_client(client):
    # Strands MCP 클라이언트에서 페이지네이션을 통해 모든 tool 가져오기
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools

Gateway로 시도해 보고 Python 클라이언트가 몇 개의 tool을 찾는지 확인하겠습니다. 
먼저 엔드포인트 URL과 JWT bearer 토큰을 기반으로 MCPClient 객체를 생성합니다. 그런 다음 
MCP 서버가 반환한 여러 tool 페이지에 걸쳐 전체 tool 세트를 검색합니다.
Given the targets we added earlier, this should return 300+ tools.

In [ ]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

이제 Gateway를 MCP Server로 사용하여 전체 tool 세트를 가져오는 3가지 방법을 보았습니다:
 

1. jsonrpc를 직접 사용
2. Strands Agent MCPClient에서 `list_tools_sync()` 메서드 사용
3. MCP Inspector tool 사용(백그라운드에서 jsonrpc 사용). 

For typical developers building agents, you'll be using option 2.

### 내장된 Gateway semantic search tool 사용
이제 MCP tool 목록에 추가되는 추가 MCP tool로 제공되는 내장 search tool을 사용하여
an additional MCP tool that gets added to your MCP tool list.

먼저 MCP를 사용하여 search tool을 실행하는 간단한 유틸리티 함수를 정의하겠습니다.
tool을 나열하는 것과 마찬가지로 gateway 엔드포인트와 JWT 토큰이 필요합니다. 그 외에는
검색 쿼리만 전달하면 됩니다. Gateway search tool이 나머지를 수행하여
matching that query against the serverless vector store that it automatically manages on your behalf.

In [ ]:
def tool_search(gateway_endpoint, jwt_token, query):
    # Gateway의 내장 semantic search tool 호출
    toolParams = {
        "name": "x_amz_bedrock_agentcore_search",
        "arguments": {"query": query},
    }
    toolResp = invoke_gateway_tool(
        gateway_endpoint=gateway_endpoint, jwt_token=jwt_token, tool_params=toolParams
    )
    tools = toolResp["result"]["structuredContent"]["tools"]
    return tools

In [ ]:
# tool search 성능 측정
start_time = time.time()
tools_found = tool_search(
    gateway_endpoint=gatewayEndpoint,
    jwt_token=jwtToken,
    query="find me 3 credit research tools",
)
end_time = time.time()
print(
    f"tool search via direct Gateway invocation took {(end_time - start_time):.2f} seconds"
)
print(f"Top tool: {tools_found[0]['name']}")

대부분의 경우 1초 이내에 검색이 얼마나 빠르게 반환되는지 확인하세요. 결과는
쿼리를 tool 메타데이터와 일치시키는 것을 기반으로 검색 관련성이 높은 순서대로 반환됩니다.
가장 관련성이 높은 tool이 목록의 첫 번째에 있습니다. search의 초기 구현은
최대 10개의 결과를 반환합니다. 그런 다음 agent에서 이러한 모든 tool을 사용하거나 가장 관련성이 높은
the most relevant matches.

# 많은 tool을 가진 MCP 서버와 Strands Agents 사용


먼저 Strands Agent와 함께 사용할 model을 선택합니다. 
이 노트북에서는 Amazon Bedrock model을 사용하지만 Strands 및 AgentCore는
can work with any LLM.

In [ ]:
# Bedrock model 설정 (Claude Haiku 4.5 사용)
bedrockmodel = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    temperature=0.7,
    streaming=True,
    boto_session=session,
)

#### agent tool로 AgentCore Gateway를 사용하는 간단한 Strands Agent
이제 AgentCore Gateway가 제공하는 MCP Server를 활용하기 위해 Strands Agent를 사용하는 것이 얼마나 쉬운지 보여드리겠습니다. 간단한

simple example, we ask the agent to add some numbers.

In [ ]:
# Gateway tool을 사용하는 간단한 Strands Agent 생성 및 실행
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    result = simple_agent("add 100 plus 50 pass ")
    print(f"{result.message['content'][0]['text']}")

Strands Agents framework를 사용하면 agent 이벤트 루프를 우회하여 MCP tool을 직접 호출할 수도 있습니다.
Gateway tool은 네이티브 MCP tool로 노출되므로 Gateway tool에 대해서도 이 작업을 수행할 수 있습니다. 여기서는
`agent.tool.<tool_name>(args)` 구문을 사용하여 Gateway MCP tool을 호출합니다:

```python
direct_result = simple_agent.tool.Calc2___add_numbers(firstNumber=10, secondNumber=20)
resp_json = json.loads(direct_result['content'][0]['text'])
```

In [ ]:
# Agent를 통하지 않고 MCP tool 직접 호출 예제
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    # agent.tool.<tool_name> 구문으로 tool 직접 호출
    direct_result = simple_agent.tool.Calc2___add_numbers(
        firstNumber=10, secondNumber=20
    )
    print(f"direct result = {direct_result}")

In [ ]:
def get_search_tool(client):
    # Gateway의 내장 search tool을 MCPAgentTool로 래핑
    mcp_tool = MCPTool(
        name="x_amz_bedrock_agentcore_search",
        description="A special tool that returns a trimmed down list of tools given a context. Use this tool only when there are many tools available and you want to get a subset that matches the provided context.",
        inputSchema={
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "search query to use for finding tools",
                }
            },
            "required": ["query"],
        },
    )
    return MCPAgentTool(mcp_tool, client)

In [ ]:
def search_using_strands(client, query):
    # Strands Agent를 사용하여 tool search 수행
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(query=query)

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    return search_results

In [ ]:
def find_strands_tools(client, query, top_n):
    # search 결과를 Strands MCPAgentTool 리스트로 변환
    strands_mcp_tools = []
    results = search_using_strands(client, query)
    for tool in results[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
# search tool을 직접 호출하여 결과 확인
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(
        query="find equity trading tools"
    )

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    print(json.dumps(search_results, indent=4))

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    results = search_using_strands(client, "find trading tools")
    print(json.dumps(search_results[0], indent=4))

    results = search_using_strands(client, "find credit research tools")
    print(json.dumps(search_results[0], indent=4))

# Strands Agent에 tool 검색 결과 추가


이제 검색에서 반환된 tool을 Strands Agent에 추가하는 방법을 살펴보겠습니다.
코딩을 더 간단하게 만들기 위해 tool 검색 결과를
Strands MCPAgentTool 객체에 매핑하는 유틸리티 함수를 제공하겠습니다. 검색 결과를 전달하고
해당 결과 중 몇 개를 agent에 전달할지
agent.

In [ ]:
def tools_to_strands_mcp_tools(tools, top_n):
    # search 결과를 Strands MCPAgentTool로 변환 (상위 N개만)
    strands_mcp_tools = []
    for tool in tools[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
# search로 찾은 tool만 사용하는 Agent 예제
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    # 사칙연산 tool 검색
    agent = Agent(
        model=bedrockmodel,
        tools=find_strands_tools(
            client,
            "tools for doing addition, subtraction, multiplication, division",
            10,
        ),
    )
    result = agent("(10*2)/(5-3)")
    print(f"{result.message['content'][0]['text']}")

In [ ]:
%%time

# search를 사용한 Agent 성능 측정
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    print("Searching for an ADDING tool from endpoint with full set of tools...")
    tools_found = tool_search(
        gateway_endpoint=gatewayEndpoint,
        jwt_token=jwtToken,
        query="tools for multiplying two numbers",
    )
    print(f"Top tool found: {tools_found[0]['name']}\n")

    agent = Agent(model=bedrockmodel, tools=tools_to_strands_mcp_tools(tools_found, 1))
    result = agent("10 * 70")
    print(f"{result.message['content'][0]['text']}")

지연 시간 개선을 확인하세요. Gateway search의 tool 하위 집합을 사용하는 이 예제는 수백 개의 tool에 의존할 때의
agent invocation when depending on hundreds of tools.

# tool search를 사용하여 3배 지연 시간 개선 시연


이제 Strands agent에서 Gateway MCP tool을 사용하는 방법과 tool을 검색하여
agent에 추가하는 방법을 알았으므로 search의 강력함을 보여드리겠습니다. 제공할 수 있는 상당한 지연 시간 감소와
입력 토큰 사용량을 강조하겠습니다.

지연 시간 및 토큰 감소를 시연하기 위해 2가지 접근 방식을 나란히 비교합니다:

1. **search 없이**. MCP 서버가 노출하는 전체 MCP tool 세트(우리의 경우 300개 이상)를 agent에 추가하고 agent가 그에 따라 tool 선택 및 호출을 수행하도록 합니다.
2. **search 사용**. 두 번째 접근 방식에서는 당면한 주제를 기반으로 검색을 수행하고 가장 관련성이 높은 tool만 agent에 보냅니다. 요점을 증명하기 위해 각각 다른 tool 세트가 필요한 두 가지 주제인 수학(숫자 더하기)과 음식(레스토랑 예약)을 사용합니다.

지연 시간 분포를 정규화하고 의미 있는 비교를 얻기 위해 각 접근 방식의
여러 반복을 수행합니다. 또한 이득을 과장하지 않기 위해 search 접근 방식을 수행할 때 agent 호출의 지연 시간뿐만 아니라
tool 검색 수행의 지연 시간도 포함합니다. 각 
반복에서 agent에게 두 가지 작업을 전달합니다: 

1. 수학 작업 - 2개의 숫자 더하기 
2. 음식 작업 - 레스토랑 예약하기

아래 결과는 이점을 보여주며 3배 지연 시간 감소와 훨씬 더 큰
입력 토큰 사용량 감소를 강조합니다. 토큰 사용량 절감이 비용 절감으로 이어지지만 입력 토큰의 상대적으로 낮은 비용으로 인해
그다지 영향을 미치지 않을 수 있습니다(많은 model provider의 경우 입력 토큰이 훨씬 저렴함). 그럼에도 불구하고 
대규모 agent 배포의 경우 입력 토큰 사용 비용도 누적될 수 있으므로 동적 search가 
agent runtime costs as well.

#### 전체 MCP tool 세트를 사용하는 agent의 지연 시간 및 토큰 사용량 측정


In [ ]:
# 모든 tool을 사용하는 Agent 성능 측정 (비교 대상)
iterations = 2
full_tokens = light_tokens = 0
full_elapsed_time = light_elapsed_time = 0

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)

In [ ]:
# 모든 tool을 사용하는 Agent 실행 (지연 시간 및 토큰 측정)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")
    heavy_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )

    math_input = "add 100 plus <iteration>"
    food_input = (
        "book me a table for 2 at Burger King under name Jo Smith at 7pm August <day>"
    )

    print("using agent with ALL tools...")
    start_time = time.time()

    for i in range(iterations):
        result = heavy_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

        result = heavy_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

    end_time = time.time()
    full_tokens = result.metrics.accumulated_usage["totalTokens"]
    full_elapsed_time = end_time - start_time
    print(f"\nTotal time: {full_elapsed_time:.1f} s, tokens: {full_tokens:,d}\n")

#### Gateway Search를 사용하는 agent의 지연 시간 및 토큰 사용량 측정
이제 동적 접근 방식을 사용하여 search를 호출하여 관련 tool을 찾은 다음 해당 관련 tool만으로
agent를 호출하겠습니다. 각 대화 턴에서 agent를 재설정하므로
conversation turn, we're also intializing the message list from conversation history of the prior turn.

In [ ]:
# search를 사용하는 Agent 실행 (지연 시간 및 토큰 측정)
with client:
    print("using agent with ONLY tools from focused search...")
    start_time = time.time()
    messages = []

    light_agent = Agent()

    for i in range(iterations):
        # 수학 작업을 위한 tool 검색
        print("Searching for an ADDING tool from endpoint with full set of tools...")
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for simply adding two numbers",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages

        # 레스토랑 예약을 위한 tool 검색
        print(
            "Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools..."
        )
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for booking a restaurant reservation",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages
        light_tokens = light_result.metrics.accumulated_usage["totalTokens"]
    end_time = time.time()

    light_elapsed_time = end_time - start_time
    print(f"\nTotal time: {light_elapsed_time:.1f} s, tokens: {light_tokens:,d}\n")

#### 결과 비교, search의 이점 강조


In [ ]:
# 성능 비교 결과 출력
print(
    f"\n\nLatency without search: {full_elapsed_time:.1f}s, using search: {light_elapsed_time:.1f}s"
)
print(f"Tokens without search: {full_tokens:,d}, using search: {light_tokens:,d}")

# 결론
이 튜토리얼에서는 Amazon Bedrock AgentCore Gateway와 내장된 
완전 관리형 semantic search 기능에 대해 배웠습니다. 다음을 확인했습니다:

- semantic search가 활성화된 gateway를 생성하는 방법
- 단일 엔드포인트에서 300개 이상의 MCP tool을 노출하기 위해 여러 gateway target을 추가하는 방법
- 3가지 다른 접근 방식을 사용하여 gateway의 tool을 나열하는 방법
- 내장된 semantic search tool을 사용하여 관련 tool을 찾는 방법
- Strands Agent와 search를 통합하는 방법
- 수백 개의 tool을 가진 서버를 사용하는 agent와 semantic search를 사용하여 tool을 특정 주제로 좁히는 agent의 성능을 비교하는 방법

AgentCore Gateway search는 더 고급 사용 사례에도 유용합니다. search를 네이티브
MCP tool로 제공하고 control plane API만이 아니라 agent에게 새로운
MCP 서버를 발견하고 런타임에 새로운 기능을 찾아 더 어려운 문제를 해결하는 데 있어 획기적인 발전을 이룰 수 있는 더 많은 자율성을 부여할 수 있습니다.
또한 search는 MCP 레지스트리의 중요한 기반이며 agent 개발자가 새로운 agent를 설계하고 구축할 때
design and build new agents.

# 리소스 정리


First let's define some helper functions for cleaning up AgentCore Gateway resources.

In [ ]:
def delete_gatewaytarget(gateway_id):
    # Gateway의 모든 target 삭제
    response = agentcore_client.list_gateway_targets(gatewayIdentifier=gateway_id)

    print(f"Found {len(response['items'])} targets for the gateway")

    for target in response["items"]:
        print(
            f"Deleting target with Name: {target['name']} and Id: {target['targetId']}"
        )

        response = agentcore_client.delete_gateway_target(
            gatewayIdentifier=gateway_id, targetId=target["targetId"]
        )
        time.sleep(20)


def delete_gateway(gateway_id):
    # Gateway 삭제
    response = agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)

### Gateway Target 삭제


In [ ]:
delete_gatewaytarget(gateway_id=gatewayId)

### Gateway 자체 삭제


In [ ]:
delete_gateway(gateway_id=gatewayId)

In [ ]:
# Lambda 함수 삭제
lambda_arns = [
    calc_lambda_resp["lambda_function_arn"],
    restaurant_lambda_resp["lambda_function_arn"],
]

for arn in lambda_arns:
    if utils.delete_gateway_lambda(arn):
        print(f"Deleted Lambda: {arn}")
    else:
        print(f"Lambda {arn} not found or deletion failed")

In [ ]:
# IAM role 및 Cognito user pool 삭제
if utils.delete_gateway_iam_role():
    print("Gateway IAM role deleted")
else:
    print("Gateway IAM role not found or deletion failed")

if utils.delete_cognito_user_pool():
    print("Cognito pool deleted")
else:
    print("✗ Failed to delete Cognito pool")